# Data Access

In [1]:
# Standard library imports
import os
import sys
from pathlib import Path
import pandas as pd
from google.oauth2 import service_account
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import numpy as np
from scipy import stats, ndimage, interpolate
from sklearn.preprocessing import StandardScaler
import google.auth
warnings.filterwarnings('ignore')  # Suppresses all warnings

# Add parent directory to Python path for local imports
notebook_path = Path.cwd()  # Gets current working directory
project_root = notebook_path.parent.parent  # Navigate up to project root
sys.path.append(str(project_root))

# Local application imports
from src.mimicdf import MIMICDF
from src.preprocessing.data_preprocessor import DataCleaner

# Initialize MIMIC database connection to GCP
mimicdf = MIMICDF.create_connection()

# Initialize MIMIC demo database
# mimicdf = MIMICDF.create_demo()

# Help
# printhelp(mimicdf)

ed_data = mimicdf.ed_data()
ed_data.sample(5)


Successfully connected to MIMIC-IV ED dataset
Loading edstays...
Table loaded: edstays
Loading demographics...
Table loaded: edstays
Table loaded: age
Loading age data...
Table loaded: patients
Calculating ED visit age...
Merging time features...
Table loaded: edstays
Merging triage features...
Table loaded: triage
Cleaning up columns...

 Dataframe shape: (425087, 18) 

Dataframe info: 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 425087 entries, 0 to 425086
Data columns (total 18 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   subject_id         425087 non-null  Int64  
 1   stay_id            425087 non-null  Int64  
 2   gender             425087 non-null  object 
 3   arrival_transport  425087 non-null  object 
 4   disposition        425087 non-null  object 
 5   race               425087 non-null  object 
 6   age_at_ed          425087 non-null  Int64  
 7   dow                425087 non-null  object 
 8   ho

,subject_id,stay_id,gender,arrival_transport,disposition,race,age_at_ed,dow,hour,minute,temperature,heartrate,resprate,o2sat,sbp,dbp,pain,acuity
155086,17073405,32725441,F,WALK IN,HOME,BLACK,51,Saturday,8,17,98.8,100.0,19.0,99.0,182.0,77.0,10,2
133265,19712154,34421659,M,WALK IN,HOME,WHITE,55,Friday,12,6,96.5,65.0,18.0,100.0,141.0,98.0,5,3
92830,16280495,34330125,M,AMBULANCE,HOME,WHITE,49,Thursday,9,46,98.0,100.0,18.0,98.0,114.0,84.0,0,2
357427,19944585,35033215,F,AMBULANCE,ADMITTED,WHITE,85,Friday,12,18,99.6,65.0,20.0,95.0,141.0,79.0,6,3
316567,15777013,30740732,M,WALK IN,ADMITTED,WHITE,70,Tuesday,18,10,97.8,58.0,16.0,100.0,168.0,86.0,3,3


## Data Quality Assessment

In [ ]:

# Basic Statistics
ed_data.describe().T


## Missingness Analysis

In [ ]:
# 1. Overall missingness by column

def initial_missingness_analysis(data):
    """
    Perform initial analysis of missingness patterns in the dataset
    """
    
    missing_summary = pd.DataFrame({
        'Missing_Count': data.isnull().sum(),
        'Missing_Percentage': (data.isnull().sum() / len(data) * 100).round(2)
    }).sort_values('Missing_Percentage', ascending=False)
    
    # 2. Visualize missingness patterns
    plt.figure(figsize=(12, 6))
    sns.barplot(x=missing_summary.index, 
                y='Missing_Percentage',
                data=missing_summary)
    plt.xticks(rotation=45, ha='right')
    plt.title('Missing Data Percentage by Variable')
    plt.ylabel('Missing Percentage')
    plt.tight_layout()
    plt.show()
    
    
    return missing_summary

# Run initial analysis
missing_summary = initial_missingness_analysis(ed_data)

In [ ]:
# # 2. Correlation heatmap only for features with missing values

def analyze_missingness(data):

    missing_features = ['age_at_ed', 'temperature', 'heartrate', 'resprate', 'o2sat', 'sbp', 'dbp', 'pain', 'acuity']
    features_with_missing = missing_features
    
    # Create binary missingness indicators only for features with missing values
    missing_binary = data[features_with_missing].isnull().astype(int)
    
    # Compute correlation between missing indicators
    missing_corr = missing_binary.corr()
    
    # Plot correlation heatmap
    plt.figure(figsize=(10, 8))
    sns.heatmap(missing_corr, 
                annot=True, 
                cmap='YlOrRd',
                fmt='.2f',
                center=0)
    plt.title('Correlation of Missingness Patterns\n(Features with Missing Values Only)')
    plt.tight_layout()
    plt.show()
    
    return missing_corr

missing_corr = analyze_missingness(ed_data)



1. Strong Correlations Among Vitals (0.75-0.98):
    - Temperature, heartrate, resprate, o2sat, sbp, and dbp show very high correlations (dark red)
    - This suggests these vitals tend to be missing together
    - Makes sense clinically as they're usually measured in the same assessment
2. Age Independence (-0.02 to -0.07):
    - Age missingness shows very weak negative correlations with vital signs
    - Suggets that age must be handled separately

In [ ]:
def missing_counts(data):
    """
    Prepares data for missing counts analysis
    """
    # Define vital signs
    vital_signs = ['temperature', 'heartrate', 'resprate', 'o2sat', 'sbp', 'dbp']
    
    # Create a copy of data to avoid modifications to original
    _df = data.copy()
    
    # Create missing indicators
    _df['missing_vitals'] = _df[vital_signs].isnull().any(axis=1).astype(int)
    _df['missing_age'] = _df['age_at_ed'].isnull().astype(int)
    _df['missing_pain'] = _df['pain'].isnull().astype(int)
    _df['missing_acuity'] = _df['acuity'].isnull().astype(int)
    
    # Keep only relevant columns
    _df = _df[['missing_vitals', 'missing_age', 'missing_pain', 'missing_acuity', 'arrival_transport', 'disposition']].reset_index(drop=True)
    
    return _df

def missing_counts_graph_annotated(data):
    """
    Creates a 2x3 grid of charts analyzing missing data patterns
    with annotations of missingness ratios.
    """
    # Process the data (assuming 'missing_counts' is a function you have defined)
    _df = missing_counts(data)

    # Create figure with more space for titles
    fig = plt.figure(figsize=(20, 12))
    
    # Create GridSpec to allow for row labels and column labels
    gs = fig.add_gridspec(3, 4, height_ratios=[1, 1, 0.1], width_ratios=[0.1, 1, 1, 1])
    
    # Create main subplot axes (2 rows, 3 columns)
    axes = np.array([
        [fig.add_subplot(gs[0, 1]), fig.add_subplot(gs[0, 2]), fig.add_subplot(gs[0, 3])],
        [fig.add_subplot(gs[1, 1]), fig.add_subplot(gs[1, 2]), fig.add_subplot(gs[1, 3])],
    ])

    # Remove spines and format all subplots
    for ax in axes.flat:
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['bottom'].set_visible(False)
        ax.spines['left'].set_visible(False)
        # Format x-axis in thousands
        ax.xaxis.set_major_formatter(lambda x, p: f'{int(x/1000)}')
        ax.grid(True, axis='x', alpha=0.75)
        # Adjust as needed
        ax.set_xlim(0, 300000)

    # Remove x-axis labels and spines for the top row
    axes[0,0].xaxis.set_ticklabels([])
    axes[0,1].xaxis.set_ticklabels([])
    axes[0,2].xaxis.set_ticklabels([])
    axes[0,0].spines['bottom'].set_visible(False)
    axes[0,1].spines['bottom'].set_visible(False)
    axes[0,2].spines['bottom'].set_visible(False)

    # Remove y-axis labels and spine for the right side
    axes[0,1].yaxis.set_visible(False)
    axes[0,1].spines['left'].set_visible(False)
    axes[0,2].yaxis.set_visible(False)
    axes[0,2].spines['left'].set_visible(False)
    axes[1,1].yaxis.set_visible(False)
    axes[1,1].spines['left'].set_visible(False)
    axes[1,2].yaxis.set_visible(False)
    axes[1,2].spines['left'].set_visible(False)

    # Add 'Count' label for bottom row
    axes[1,0].set_xlabel('Count (Thousands)')
    axes[1,1].set_xlabel('Count (Thousands)')
    axes[1,2].set_xlabel('Count (Thousands)')

    # Add row labels (rotated 90 degrees)
    row_labels = ['Arrival', 'Disposition']
    for idx, label in enumerate(row_labels):
        fig.add_subplot(gs[idx, 0]).text(0.5, 0.5, label, 
                                         rotation=90, 
                                         ha='center', 
                                         va='center',
                                         fontsize=18,
                                         fontweight='bold')
        plt.axis('off')  # Hide the axis for the label subplots

    # Add column labels at the bottom
    col_labels = ['Missing Vitals', 'Missing Acuity', 'Missing Pain']
    for idx, label in enumerate(col_labels):
        fig.add_subplot(gs[2, idx+1]).text(0.5, 0.5, label,
                                           ha='center',
                                           va='center',
                                           fontsize=18,
                                           fontweight='bold')
        plt.axis('off')

    # Add title and subtitle
    fig.suptitle('Missing Data Analysis in Emergency Department Records', 
                 y=0.95, fontsize=22, weight='bold')
    plt.figtext(0.5, 0.91, 
                'Distribution of missing vital signs, age, and pain data across arrival modes and dispositions',
                ha='center', fontsize=18)

    # Helper function to convert to camel case (for y-tick labels)
    def to_camel_case(text):
        words = text.replace('_', ' ').title().split()
        camel_case = words[0] + ' ' + ' '.join(word.capitalize() for word in words[1:])
        return camel_case if len(camel_case) <= 15 else camel_case[:11] + '...'

    # 1) Missing Vitals by arrival_transport (top-left)
    transport_counts = _df.groupby('arrival_transport').size()
    missing_vitals_counts = _df.groupby('arrival_transport')['missing_vitals'].sum()
    sort_idx = transport_counts.sort_values(ascending=True).index
    transport_counts = transport_counts[sort_idx]
    missing_vitals_counts = missing_vitals_counts[sort_idx]
    
    ax = axes[0,0]
    y_pos = np.arange(len(transport_counts))

    ax.barh(y_pos, transport_counts, color='lightgray', alpha=0.5, label='Total Count')
    bars_vitals = ax.barh(y_pos, missing_vitals_counts, color='skyblue', alpha=0.7, label='Missing Vitals')
    ax.set_yticks(y_pos)
    ax.set_yticklabels([to_camel_case(str(x)) for x in sort_idx])

    # Annotate missing_vitals bars with ratio (no decimals, e.g. 25%)
    for i, bar in enumerate(bars_vitals):
        total_val = transport_counts.iloc[i]
        missing_val = missing_vitals_counts.iloc[i]
        if total_val > 0:
            ratio = int((missing_val / total_val) * 100)  # no decimals
            x_bar = bar.get_width()
            y_bar = bar.get_y() + bar.get_height() / 2
            # Place text just to the right of the bar
            ax.text(x_bar + 5000,
                    y_bar,
                    f"{ratio}%",
                    va='center', ha='left',
                    color='black',
                    fontsize=11,
                    fontweight='bold')

    # 2) Missing Vitals by disposition (bottom-left)
    disposition_counts = _df.groupby('disposition').size()
    missing_vitals_counts = _df.groupby('disposition')['missing_vitals'].sum()
    sort_idx = disposition_counts.sort_values(ascending=True).index
    disposition_counts = disposition_counts[sort_idx]
    missing_vitals_counts = missing_vitals_counts[sort_idx]
    
    ax = axes[1,0]
    y_pos = np.arange(len(disposition_counts))

    ax.barh(y_pos, disposition_counts, color='lightgray', alpha=0.5, label='Total Count')
    bars_vitals = ax.barh(y_pos, missing_vitals_counts, color='skyblue', alpha=0.7, label='Missing Vitals')
    ax.set_yticks(y_pos)
    ax.set_yticklabels([to_camel_case(str(x)) for x in sort_idx])

    # Annotate missing_vitals bars with ratio
    for i, bar in enumerate(bars_vitals):
        total_val = disposition_counts.iloc[i]
        missing_val = missing_vitals_counts.iloc[i]
        if total_val > 0:
            ratio = int((missing_val / total_val) * 100)
            x_bar = bar.get_width()
            y_bar = bar.get_y() + bar.get_height() / 2
            ax.text(x_bar + 5000,
                    y_bar,
                    f"{ratio}%",
                    va='center', ha='left',
                    color='black',
                    fontsize=11,
                    fontweight='bold')

    # 3) Missing Acuity by arrival_transport (top-middle)
    arrival_transport_counts = _df.groupby('arrival_transport').size()
    missing_acuity_counts = _df.groupby('arrival_transport')['missing_acuity'].sum()
    sort_idx = arrival_transport_counts.sort_values(ascending=True).index
    arrival_transport_counts = arrival_transport_counts[sort_idx]
    missing_acuity_counts = missing_acuity_counts[sort_idx]
    
    ax = axes[0,1]
    y_pos = np.arange(len(missing_acuity_counts))

    ax.barh(y_pos, arrival_transport_counts, color='lightgray', alpha=0.5, label='Total Count')
    bars_acuity = ax.barh(y_pos, missing_acuity_counts, color='lightcoral', alpha=0.7, label='Missing Acuity')

    # Annotate missing_acuity bars with ratio
    for i, bar in enumerate(bars_acuity):
        total_val = arrival_transport_counts.iloc[i]
        missing_val = missing_acuity_counts.iloc[i]
        if total_val > 0:
            ratio = int((missing_val / total_val) * 100)
            x_bar = bar.get_width()
            y_bar = bar.get_y() + bar.get_height() / 2
            ax.text(x_bar + 5000,
                    y_bar,
                    f"{ratio}%",
                    va='center', ha='left',
                    color='black',
                    fontsize=11,
                    fontweight='bold')  

    # 4) Missing Acuity by disposition (bottom-middle)
    disposition_counts = _df.groupby('disposition').size()
    missing_acuity_counts = _df.groupby('disposition')['missing_acuity'].sum()
    sort_idx = disposition_counts.sort_values(ascending=True).index
    disposition_counts = disposition_counts[sort_idx]
    missing_acuity_counts = missing_acuity_counts[sort_idx]

    ax = axes[1,1]
    y_pos = np.arange(len(missing_acuity_counts))

    ax.barh(y_pos, disposition_counts, color='lightgray', alpha=0.5, label='Total Count')
    bars_acuity = ax.barh(y_pos, missing_acuity_counts, color='lightcoral', alpha=0.7, label='Missing Acuity')
    
    # Annotate missing_acuity bars with ratio
    for i, bar in enumerate(bars_acuity):
        total_val = disposition_counts.iloc[i]
        missing_val = missing_acuity_counts.iloc[i]
        if total_val > 0:
            ratio = int((missing_val / total_val) * 100)
            x_bar = bar.get_width()
            y_bar = bar.get_y() + bar.get_height() / 2
            ax.text(x_bar + 5000,
                    y_bar,
                    f"{ratio}%",
                    va='center', ha='left',
                    color='black',
                    fontsize=11,
                    fontweight='bold')

    # 5) Missing Pain by arrival_transport (top-right)
    arrival_transport_counts = _df.groupby('arrival_transport').size()
    missing_pain_counts = _df.groupby('arrival_transport')['missing_pain'].sum()
    sort_idx = arrival_transport_counts.sort_values(ascending=True).index
    arrival_transport_counts = arrival_transport_counts[sort_idx]
    missing_pain_counts = missing_pain_counts[sort_idx]
    
    ax = axes[0,2]
    y_pos = np.arange(len(missing_pain_counts))

    ax.barh(y_pos, arrival_transport_counts, color='lightgray', alpha=0.5, label='Total Count')
    bars_pain = ax.barh(y_pos, missing_pain_counts, color='plum', alpha=0.7, label='Missing Pain')
    
    # Annotate missing_pain bars with ratio
    for i, bar in enumerate(bars_pain):
        total_val = arrival_transport_counts.iloc[i]
        missing_val = missing_pain_counts.iloc[i]
        if total_val > 0:
            ratio = int((missing_val / total_val) * 100)
            x_bar = bar.get_width()
            y_bar = bar.get_y() + bar.get_height() / 2
            ax.text(x_bar + 5000,
                    y_bar,
                    f"{ratio}%",
                    va='center', ha='left',
                    color='black',
                    fontsize=11,
                    fontweight='bold')

    # 6) Missing Pain by disposition (bottom-right)
    disposition_counts = _df.groupby('disposition').size()
    missing_pain_counts = _df.groupby('disposition')['missing_pain'].sum()
    sort_idx = disposition_counts.sort_values(ascending=True).index
    disposition_counts = disposition_counts[sort_idx]
    missing_pain_counts = missing_pain_counts[sort_idx]
    
    ax = axes[1,2]
    y_pos = np.arange(len(missing_pain_counts))

    ax.barh(y_pos, disposition_counts, color='lightgray', alpha=0.5, label='Total Count')
    bars_pain = ax.barh(y_pos, missing_pain_counts, color='plum', alpha=0.7, label='Missing Pain')
    
    # Annotate missing_pain bars with ratio
    for i, bar in enumerate(bars_pain):
        total_val = disposition_counts.iloc[i]
        missing_val = missing_pain_counts.iloc[i]
        if total_val > 0:
            ratio = int((missing_val / total_val) * 100)
            x_bar = bar.get_width()
            y_bar = bar.get_y() + bar.get_height() / 2
            ax.text(x_bar + 5000,
                    y_bar,
                    f"{ratio}%",
                    va='center', ha='left',
                    color='black',
                    fontsize=11,
                    fontweight='bold')

    plt.tight_layout()
    # Adjust layout to make room for the main title
    plt.subplots_adjust(top=0.85)
    plt.show()

missing_counts_graph_annotated(ed_data)

In [8]:
def plot_features_distribution(ed_data):
    plt.style.use('default')  # Using default style instead of seaborn
    features = ['age_at_ed', 'temperature', 'heartrate', 'resprate', 'o2sat', 'sbp', 'dbp', 'pain', 'acuity']
    n_features = len(features)
    n_rows = (n_features + 2) // 3  # Calculate number of rows needed

    fig, axes = plt.subplots(n_rows, 3, figsize=(15, 5*n_rows))
    axes = axes.ravel()  # Flatten axes array for easier indexing

    for idx, feature in enumerate(features):
        # Create boxplot and histogram side by side
        ax = axes[idx]
        
        # Create violin plot with boxplot inside
        sns.violinplot(data=ed_data, y=feature, ax=ax, inner='box', color='lightblue')
        
        # Add title and labels
        ax.set_title(f'{feature} Distribution')
        ax.set_ylabel(feature)
        
        # Add text with basic statistics
        stats_text = f'Mean: {ed_data[feature].mean():.2f}\n'
        stats_text += f'Median: {ed_data[feature].median():.2f}\n'
        stats_text += f'Missing: {ed_data[feature].isna().sum()/len(ed_data)*100:.1f}%'
        ax.text(0.95, 0.95, stats_text,
                transform=ax.transAxes,
                verticalalignment='top',
                horizontalalignment='right',
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

    # Remove any empty subplots
    for idx in range(n_features, len(axes)):
        fig.delaxes(axes[idx])

    plt.tight_layout()
    plt.show()

# Call the function with ed_data
# plot_features_distribution(ed_data)



## Data Preprocessing

In [2]:
def plot_features_distribution(ed_data):
    plt.style.use('default')  # Using default style instead of seaborn
    features = ['age_at_ed', 'temperature', 'heartrate', 'resprate', 'o2sat', 'sbp', 'dbp', 'pain', 'acuity']
    n_features = len(features)
    n_rows = (n_features + 2) // 3  # Calculate number of rows needed

    fig, axes = plt.subplots(n_rows, 3, figsize=(15, 5*n_rows))
    axes = axes.ravel()  # Flatten axes array for easier indexing

    for idx, feature in enumerate(features):
        # Create boxplot and histogram side by side
        ax = axes[idx]
        
        # Create violin plot with boxplot inside
        sns.violinplot(data=ed_data, y=feature, ax=ax, inner='box', color='lightblue')
        
        # Add title and labels
        ax.set_title(f'{feature} Distribution')
        ax.set_ylabel(feature)
        
        # Add text with basic statistics
        stats_text = f'Mean: {ed_data[feature].mean():.2f}\n'
        stats_text += f'Median: {ed_data[feature].median():.2f}\n'
        stats_text += f'Missing: {ed_data[feature].isna().sum()/len(ed_data)*100:.1f}%'
        ax.text(0.95, 0.95, stats_text,
                transform=ax.transAxes,
                verticalalignment='top',
                horizontalalignment='right',
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

    # Remove any empty subplots
    for idx in range(n_features, len(axes)):
        fig.delaxes(axes[idx])

    plt.tight_layout()
    plt.show()

# Call the function with ed_data
# plot_features_distribution(ed_data)


# Preprocess ed_data to handle invlaid values
data_preprocessor = DataCleaner(ed_data)
cleaned_data = data_preprocessor.prepare_data()

plot_features_distribution(cleaned_data)
cleaned_data.describe().T

NameError: name 'DataCleaner' is not defined

In [ ]:
# Create figure with two subplots side by side
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Distribution plot
sns.histplot(data=cleaned_data, x='los_minutes', bins=50, ax=ax1)
ax1.set_title('Distribution of Length of Stay')
ax1.set_xlabel('Length of Stay (minutes)')
ax1.set_ylabel('Count')

# Add mean and median lines
mean_los = cleaned_data['los_minutes'].mean()
median_los = cleaned_data['los_minutes'].median()
ax1.axvline(mean_los, color='red', linestyle='--', label=f'Mean: {mean_los:.1f}')
ax1.axvline(median_los, color='green', linestyle='--', label=f'Median: {median_los:.1f}')
ax1.legend()

# Q-Q plot
stats.probplot(cleaned_data['los_minutes'], dist="norm", plot=ax2)
ax2.set_title('Q-Q Plot of Length of Stay')

plt.tight_layout()
plt.show()

# Print summary statistics
print("\nSummary Statistics for Length of Stay (minutes):")
print(cleaned_data['los_minutes'].describe())

In [ ]:
# Calculate IQR-based bounds
Q1 = cleaned_data['los_minutes'].quantile(0.25)
Q3 = cleaned_data['los_minutes'].quantile(0.75)
IQR = Q3 - Q1

# Set lower and upper bounds
# Lower bound: 0 minutes (can't have negative LOS)
# Upper bound: Q3 + 1.5*IQR (standard outlier definition)
lower_bound = 0
upper_bound = Q3 + 1.5 * IQR

# Filter data
filtered_data = cleaned_data[
    (cleaned_data['los_minutes'] >= lower_bound) & 
    (cleaned_data['los_minutes'] <= upper_bound)
]

# Print statistics
print(f"Original data points: {len(cleaned_data)}")
print(f"Filtered data points: {len(filtered_data)}")
print(f"Removed {len(cleaned_data) - len(filtered_data)} outliers")
print(f"\nUpper bound: {upper_bound:.1f} minutes ({upper_bound/60:.1f} hours)")

# Plot new distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Distribution plot
sns.histplot(data=filtered_data, x='los_minutes', bins=50, ax=ax1)
ax1.set_title('Distribution of Length of Stay (Filtered)')
ax1.set_xlabel('Length of Stay (minutes)')
ax1.set_ylabel('Count')

# Add mean and median lines
mean_los = filtered_data['los_minutes'].mean()
median_los = filtered_data['los_minutes'].median()
ax1.axvline(mean_los, color='red', linestyle='--', label=f'Mean: {mean_los:.1f}')
ax1.axvline(median_los, color='green', linestyle='--', label=f'Median: {median_los:.1f}')
ax1.legend()

# Q-Q plot
stats.probplot(filtered_data['los_minutes'], dist="norm", plot=ax2)
ax2.set_title('Q-Q Plot of Length of Stay (Filtered)')

plt.tight_layout()
plt.show()

In [ ]:
# Calculate both IQR and STD bounds
mean = cleaned_data['los_minutes'].mean()
std = cleaned_data['los_minutes'].std()

# Set bounds
# STD method: mean ± 3 standard deviations (99.7% of data in normal distribution)
std_lower = mean - 3 * std
std_upper = mean + 3 * std
# Ensure lower bound isn't negative
std_lower = max(0, std_lower)

# Filter data using STD method
std_filtered_data = cleaned_data[
    (cleaned_data['los_minutes'] >= std_lower) & 
    (cleaned_data['los_minutes'] <= std_upper)
]

# Create figure with two subplots side by side
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Distribution plot
sns.histplot(data=std_filtered_data, x='los_minutes', bins=50, ax=ax1)
ax1.set_title('Distribution of Length of Stay (STD Filtered)')
ax1.set_xlabel('Length of Stay (minutes)')
ax1.set_ylabel('Count')

# Add mean and median lines
mean_los = std_filtered_data['los_minutes'].mean()
median_los = std_filtered_data['los_minutes'].median()
ax1.axvline(mean_los, color='red', linestyle='--', label=f'Mean: {mean_los:.1f}')
ax1.axvline(median_los, color='green', linestyle='--', label=f'Median: {median_los:.1f}')
ax1.legend()

# Q-Q plot
stats.probplot(std_filtered_data['los_minutes'], dist="norm", plot=ax2)
ax2.set_title('Q-Q Plot of Length of Stay (STD Filtered)')

plt.tight_layout()
plt.show()

# Print statistics
print(f"Original data points: {len(cleaned_data)}")
print(f"STD filtered data points: {len(std_filtered_data)}")
print(f"Removed {len(cleaned_data) - len(std_filtered_data)} outliers")
print(f"\nUpper bound: {std_upper:.1f} minutes ({std_upper/60:.1f} hours)")
print(f"Lower bound: {std_lower:.1f} minutes ({std_lower/60:.1f} hours)")

In [ ]:
sample_data = cleaned_data.sample(100)
# sns pairplot of all columns in cleaned_data with kde, hue = disposition
sns.pairplot(sample_data, hue='disposition', kind='kde')
plt.show()

# sns pairplot of all columns in cleaned_data with kde, hue = arrival_transport
sns.pairplot(sample_data, hue='arrival_transport', kind='kde')
plt.show()

In [ ]:
# Import necessary libraries
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
import numpy as np

# Select numerical features for clustering
numerical_features = ['age_at_ed', 'temperature', 'heartrate', 'resprate', 
                     'o2sat', 'sbp', 'dbp', 'pain', 'acuity']

# Prepare the data
X = cleaned_data[numerical_features]

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Initialize and fit GMM
n_components = 5  # You can adjust this number
gmm = GaussianMixture(n_components=n_components, 
                      random_state=42,
                      covariance_type='full',
                      n_init=10)
gmm.fit(X_scaled)

# Get cluster assignments
clusters = gmm.predict(X_scaled)

# Add cluster labels to the original dataframe
cleaned_data['cluster'] = clusters

# Analyze cluster characteristics
cluster_stats = cleaned_data.groupby('cluster')[numerical_features].agg(['mean', 'std'])

# Visualize results
plt.figure(figsize=(15, 6))

# Plot 1: Cluster sizes
plt.subplot(1, 2, 1)
cluster_sizes = cleaned_data['cluster'].value_counts().sort_index()
plt.bar(cluster_sizes.index, cluster_sizes.values)
plt.title('Cluster Sizes')
plt.xlabel('Cluster')
plt.ylabel('Number of Samples')

# Plot 2: BIC scores for different numbers of components
plt.subplot(1, 2, 2)
n_components_range = range(1, 11)
bic = []
for n in n_components_range:
    gmm_temp = GaussianMixture(n_components=n, random_state=42, n_init=10)
    gmm_temp.fit(X_scaled)
    bic.append(gmm_temp.bic(X_scaled))
    
plt.plot(n_components_range, bic, marker='o')
plt.title('BIC Score vs. Number of Components')
plt.xlabel('Number of Components')
plt.ylabel('BIC Score')

plt.tight_layout()
plt.show()

# Print cluster statistics
print("\nCluster Statistics:")
print(cluster_stats)

In [ ]:
cluster_stats

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import itertools
import numpy as np

def plot_cluster_circles_from_stats(cluster_stats, feature_names):
    # Create pairs of features
    feature_pairs = list(itertools.combinations(range(len(feature_names)), 2))
    
    # Calculate number of rows and columns needed
    n_pairs = len(feature_pairs)
    n_cols = 3  # You can adjust this
    n_rows = (n_pairs + n_cols - 1) // n_cols
    
    # Create figure
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))
    axes = axes.ravel()
    
    # Colors for clusters
    n_clusters = len(cluster_stats.index)
    colors = plt.cm.viridis(np.linspace(0, 1, n_clusters))
    
    # For each pair of features
    for idx, (i, j) in enumerate(feature_pairs):
        ax = axes[idx]
        
        feature_i = feature_names[i]
        feature_j = feature_names[j]
        
        # Plot circles for each cluster
        for k in range(n_clusters):
            # Get mean and std for these features
            mean_i = cluster_stats.iloc[k][f'{feature_i}']['mean']
            mean_j = cluster_stats.iloc[k][f'{feature_j}']['mean']
            std_i = cluster_stats.iloc[k][f'{feature_i}']['std']
            std_j = cluster_stats.iloc[k][f'{feature_j}']['std']
            
            # Create circle
            circle = patches.Ellipse((mean_i, mean_j), 
                                   width=2*std_i, 
                                   height=2*std_j,
                                   alpha=0.3,
                                   color=colors[k],
                                   label=f'Cluster {k}')
            ax.add_patch(circle)
            
            # Add cluster center
            ax.plot(mean_i, mean_j, 'o', 
                   color=colors[k], 
                   markersize=10,
                   markeredgecolor='black')
        
        # Set labels
        ax.set_xlabel(feature_i)
        ax.set_ylabel(feature_j)
        
        # Add legend to first plot only
        if idx == 0:
            ax.legend()
            
        # Set title
        ax.set_title(f'{feature_i} vs {feature_j}')
        
        # Make plot square
        ax.set_aspect('equal', adjustable='box')
    
    # Remove empty subplots
    for idx in range(len(feature_pairs), len(axes)):
        fig.delaxes(axes[idx])
    
    plt.tight_layout()
    plt.show()

# Define feature names
feature_names = ['age_at_ed', 'temperature', 'heartrate', 'resprate', 
                'o2sat', 'sbp', 'dbp', 'pain', 'acuity']

# Create visualization
plot_cluster_circles_from_stats(cluster_stats, feature_names)

# You might also want to see just a few key relationships
key_features = ['age_at_ed', 'heartrate', 'resprate', 'pain']
plot_cluster_circles_from_stats(cluster_stats, key_features)

In [ ]:
# Import necessary libraries
import hdbscan
from sklearn.preprocessing import StandardScaler
import numpy as np

# Select numerical features for clustering
numerical_features = ['age_at_ed', 'temperature', 'heartrate', 'resprate', 
                     'o2sat', 'sbp', 'dbp', 'pain', 'acuity']

# Sample data and prepare for clustering
sample_size = 10000
sampled_data = cleaned_data.sample(n=sample_size, random_state=42)
X = sampled_data[numerical_features]

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Initialize and fit HDBSCAN
clusterer = hdbscan.HDBSCAN(
    min_cluster_size=50,  # Minimum size of clusters
    min_samples=5,        # Number of samples in neighborhood for core points
    metric='euclidean',
    cluster_selection_method='eom'  # 'eom' tends to produce more clusters than 'leaf'
)
clusters = clusterer.fit_predict(X_scaled)

# Add cluster labels to the sampled dataframe
sampled_data['cluster'] = clusters

# Visualize results
plt.figure(figsize=(15, 6))

# Plot cluster sizes
cluster_sizes = sampled_data['cluster'].value_counts().sort_index()
plt.bar(cluster_sizes.index, cluster_sizes.values)
plt.title('Cluster Sizes')
plt.xlabel('Cluster (-1 represents noise points)')
plt.ylabel('Number of Samples')
plt.tight_layout()
plt.show()

# Print cluster statistics
print("\nCluster Statistics:")
cluster_stats = sampled_data.groupby('cluster')[numerical_features].agg(['mean', 'std'])
print(cluster_stats)

# Optional: Plot feature distributions by cluster
for feature in numerical_features:
    plt.figure(figsize=(10, 6))
    for cluster in sorted(sampled_data['cluster'].unique()):
        cluster_data = sampled_data[sampled_data['cluster'] == cluster][feature]
        plt.hist(cluster_data, alpha=0.5, label=f'Cluster {cluster}', bins=30)
    plt.title(f'{feature} Distribution by Cluster')
    plt.xlabel(feature)
    plt.ylabel('Count')
    plt.legend()
    plt.show()

In [ ]:
# Import necessary libraries
import hdbscan
from sklearn.preprocessing import StandardScaler
import numpy as np

# Select numerical features for clustering - removing ordinal variables
numerical_features = ['age_at_ed', 'temperature', 'heartrate', 'resprate', 
                     'o2sat', 'sbp', 'dbp']

# Sample data and prepare for clustering
sample_size = 10000
sampled_data = cleaned_data.sample(n=sample_size, random_state=42)
X = sampled_data[numerical_features]

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Try different HDBSCAN configurations
configs = [
    {'min_cluster_size': 100, 'min_samples': 5},
    {'min_cluster_size': 200, 'min_samples': 10},
    {'min_cluster_size': 50, 'min_samples': 2},
    {'min_cluster_size': 30, 'min_samples': 1}
]

for config in configs:
    # Initialize and fit HDBSCAN with current config
    clusterer = hdbscan.HDBSCAN(
        min_cluster_size=config['min_cluster_size'],
        min_samples=config['min_samples'],
        metric='euclidean',
        cluster_selection_method='eom',
        cluster_selection_epsilon=0.5  # More permissive cluster selection
    )
    clusters = clusterer.fit_predict(X_scaled)
    
    # Calculate noise percentage
    noise_percentage = (clusters == -1).mean() * 100
    
    # Plot cluster sizes
    plt.figure(figsize=(15, 6))
    cluster_sizes = pd.Series(clusters).value_counts().sort_index()
    plt.bar(cluster_sizes.index, cluster_sizes.values)
    plt.title(f'Cluster Sizes\nConfig: {config}\nNoise: {noise_percentage:.1f}%')
    plt.xlabel('Cluster (-1 represents noise points)')
    plt.ylabel('Number of Samples')
    plt.tight_layout()
    plt.show()
    
    if noise_percentage < 50:  # Only show detailed stats if we have reasonable clustering
        # Add cluster labels to a copy of sampled data
        temp_data = sampled_data.copy()
        temp_data['cluster'] = clusters
        
        # Print cluster statistics
        print(f"\nCluster Statistics for config {config}:")
        cluster_stats = temp_data.groupby('cluster')[numerical_features].agg(['mean', 'std'])
        print(cluster_stats)
        
        # Plot feature distributions for non-noise clusters
        for feature in numerical_features:
            plt.figure(figsize=(10, 6))
            for cluster in sorted(set(clusters[clusters != -1])):  # Skip noise cluster
                cluster_data = temp_data[temp_data['cluster'] == cluster][feature]
                plt.hist(cluster_data, alpha=0.5, label=f'Cluster {cluster}', bins=30)
            plt.title(f'{feature} Distribution by Cluster\nConfig: {config}')
            plt.xlabel(feature)
            plt.ylabel('Count')
            plt.legend()
            plt.show()

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Select numerical features for clustering
numerical_features = ['age_at_ed', 'temperature', 'heartrate', 'resprate', 
                     'o2sat', 'sbp', 'dbp']

# Sample data and prepare for clustering
sample_size = 10000
sampled_data = cleaned_data.sample(n=sample_size, random_state=42)
X = sampled_data[numerical_features]

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Try different numbers of clusters
silhouette_scores = []
k_range = range(2, 11)

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    clusters = kmeans.fit_predict(X_scaled)
    score = silhouette_score(X_scaled, clusters)
    silhouette_scores.append(score)
    
# Plot silhouette scores
plt.figure(figsize=(10, 6))
plt.plot(k_range, silhouette_scores, 'bo-')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Score vs Number of Clusters')
plt.show()

# Use optimal k from silhouette analysis
optimal_k = k_range[np.argmax(silhouette_scores)]
kmeans = KMeans(n_clusters=optimal_k, random_state=42)
clusters = kmeans.fit_predict(X_scaled)

# Add cluster labels to the sampled dataframe
sampled_data['cluster'] = clusters

# Plot cluster sizes
plt.figure(figsize=(15, 6))
cluster_sizes = sampled_data['cluster'].value_counts().sort_index()
plt.bar(cluster_sizes.index, cluster_sizes.values)
plt.title(f'Cluster Sizes (k={optimal_k})')
plt.xlabel('Cluster')
plt.ylabel('Number of Samples')
plt.show()

# Print cluster statistics
print("\nCluster Statistics:")
cluster_stats = sampled_data.groupby('cluster')[numerical_features].agg(['mean', 'std'])
print(cluster_stats)

# Plot feature distributions by cluster
for feature in numerical_features:
    plt.figure(figsize=(10, 6))
    for cluster in sorted(sampled_data['cluster'].unique()):
        cluster_data = sampled_data[sampled_data['cluster'] == cluster][feature]
        plt.hist(cluster_data, alpha=0.5, label=f'Cluster {cluster}', bins=30)
    plt.title(f'{feature} Distribution by Cluster')
    plt.xlabel(feature)
    plt.ylabel('Count')
    plt.legend()
    plt.show()

# Analyze how clusters relate to acuity
print("\nAcuity Distribution by Cluster:")
acuity_dist = pd.crosstab(sampled_data['cluster'], sampled_data['acuity'], normalize='index') * 100
print(acuity_dist)

# Visualize cluster centers
cluster_centers = pd.DataFrame(
    scaler.inverse_transform(kmeans.cluster_centers_),
    columns=numerical_features
)
print("\nCluster Centers (Original Scale):")
print(cluster_centers)

In [ ]:
# Use the same sampled data and scaled features from previous analysis
# Try different numbers of components and covariance types
n_components_range = range(2, 11)
covariance_types = ['full', 'tied', 'diag', 'spherical']

# Initialize storage for BIC and AIC scores
bic_scores = np.zeros((len(covariance_types), len(n_components_range)))
aic_scores = np.zeros((len(covariance_types), len(n_components_range)))

# Compute scores for each combination
for i, cv_type in enumerate(covariance_types):
    for j, n_comp in enumerate(n_components_range):
        gmm = GaussianMixture(
            n_components=n_comp,
            covariance_type=cv_type,
            random_state=42,
            n_init=10
        )
        gmm.fit(X_scaled)
        bic_scores[i, j] = gmm.bic(X_scaled)
        aic_scores[i, j] = gmm.aic(X_scaled)

# Plot BIC and AIC scores
plt.figure(figsize=(15, 6))

# BIC plot
plt.subplot(1, 2, 1)
for i, cv_type in enumerate(covariance_types):
    plt.plot(n_components_range, bic_scores[i], label=cv_type, marker='o')
plt.xlabel('Number of components')
plt.ylabel('BIC score')
plt.title('BIC Score vs. Number of Components')
plt.legend()

# AIC plot
plt.subplot(1, 2, 2)
for i, cv_type in enumerate(covariance_types):
    plt.plot(n_components_range, aic_scores[i], label=cv_type, marker='o')
plt.xlabel('Number of components')
plt.ylabel('AIC score')
plt.title('AIC Score vs. Number of Components')
plt.legend()

plt.tight_layout()
plt.show()

# Find optimal parameters
best_bic_idx = np.unravel_index(bic_scores.argmin(), bic_scores.shape)
best_cv_type = covariance_types[best_bic_idx[0]]
best_n_components = n_components_range[best_bic_idx[1]]

print(f"\nOptimal parameters based on BIC:")
print(f"Covariance type: {best_cv_type}")
print(f"Number of components: {best_n_components}")

# Fit optimal GMM
optimal_gmm = GaussianMixture(
    n_components=best_n_components,
    covariance_type=best_cv_type,
    random_state=42,
    n_init=10
)
clusters = optimal_gmm.fit_predict(X_scaled)

# Add cluster labels to the sampled dataframe
sampled_data['cluster'] = clusters

# Plot cluster sizes
plt.figure(figsize=(15, 6))
cluster_sizes = sampled_data['cluster'].value_counts().sort_index()
plt.bar(cluster_sizes.index, cluster_sizes.values)
plt.title(f'GMM Cluster Sizes (n={best_n_components}, covariance={best_cv_type})')
plt.xlabel('Cluster')
plt.ylabel('Number of Samples')
plt.show()

# Print cluster statistics
print("\nCluster Statistics:")
cluster_stats = sampled_data.groupby('cluster')[numerical_features].agg(['mean', 'std'])
print(cluster_stats)

# Plot feature distributions by cluster
for feature in numerical_features:
    plt.figure(figsize=(10, 6))
    for cluster in sorted(sampled_data['cluster'].unique()):
        cluster_data = sampled_data[sampled_data['cluster'] == cluster][feature]
        plt.hist(cluster_data, alpha=0.5, label=f'Cluster {cluster}', bins=30)
    plt.title(f'{feature} Distribution by Cluster')
    plt.xlabel(feature)
    plt.ylabel('Count')
    plt.legend()
    plt.show()

# Analyze cluster probabilities
cluster_probs = optimal_gmm.predict_proba(X_scaled)
avg_prob = np.mean(np.max(cluster_probs, axis=1))
print(f"\nAverage probability of cluster assignment: {avg_prob:.3f}")

# Analyze how clusters relate to acuity
print("\nAcuity Distribution by Cluster:")
acuity_dist = pd.crosstab(sampled_data['cluster'], sampled_data['acuity'], normalize='index') * 100
print(acuity_dist)

# Get cluster centers
cluster_centers = pd.DataFrame(
    scaler.inverse_transform(optimal_gmm.means_),
    columns=numerical_features
)
print("\nCluster Centers (Original Scale):")
print(cluster_centers)